In [11]:
import torch
from torchinfo import summary
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [12]:
image_path = 'dataset'
transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = torchvision.datasets.MNIST(root= image_path, train= True,
transform= transform, download= True)
mnist_valid_dataset = Subset(mnist_dataset,
torch.arange(1000))
mnist_train_dataset = Subset(mnist_dataset,
torch.arange(1000, 11000))
#torch.arange(1000, len(mnist_dataset)))
mnist_test_dataset = torchvision.datasets.MNIST(root= image_path, train= False,
transform= transform, download=False)

In [13]:
batch_size = 32
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size= batch_size, shuffle=True)
valid_dl = DataLoader(mnist_valid_dataset, batch_size= batch_size, shuffle=False)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Esta implementación utilizará {device} para el entrenamiento e inferencia del modelo')

Esta implementación utilizará cpu para el entrenamiento e inferencia del modelo


In [15]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10

In [16]:
class CustomRNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int,num_classes: int, sequence_length: int):
        super(CustomRNN, self).__init__()
        # Hiperparámetros
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.sequence_length = sequence_length
        # Capas
        self.rnn1 = nn.RNN(input_size, hidden_size, num_layers,batch_first=True)
        self.rnn2 = nn.RNN(hidden_size, hidden_size, num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        def forward(self, x):
            x = x.reshape(-1, self.sequence_length, self.input_size)
            out, hidden = self.rnn1(x)
            out, hidden = self.rnn2(out)
            out = self.fc(out[:, -1, :])
            return out
model = CustomRNN(input_size, hidden_size, num_layers, num_classes,sequence_length)
model.to(device)

CustomRNN(
  (rnn1): RNN(28, 128, batch_first=True)
  (rnn2): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [17]:
summary(model,input_data= torch.randint(0, 255, (batch_size, 1, 28, 28))/ 255,col_names=["output_size", "num_params"], verbose= 0,device= device)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [18]:
lr = 0.001
num_epochs = 15

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= .001)

In [21]:
#Librerias
from tqdm.notebook import tqdm
#Entrenamiento
def train(model, num_epochs, train_dl, valid_dl, device):
    loss_hist_train = torch.zeros(num_epochs).to(device)
    accuracy_hist_train = torch.zeros(num_epochs).to(device)
    loss_hist_valid = torch.zeros(num_epochs).to(device)
    accuracy_hist_valid = torch.zeros(num_epochs).to(device)
    for epoch in range(num_epochs):
        with tqdm(train_dl, unit="batch") as tepoch:
            model.train()
            for x_batch, y_batch in tepoch:
                tepoch.set_description(f"Epoch {epoch} train")
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss_hist_train[epoch] += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                
                accuracy_hist_train[epoch] += is_correct.sum()
                tepoch.set_postfix(loss= loss_hist_train[epoch].item() /len(train_dl.dataset),accuracy= 100. * accuracy_hist_train[epoch].item() / len(train_dl.dataset))
            loss_hist_train[epoch] /= len(train_dl.dataset)
            accuracy_hist_train[epoch] /= len(train_dl.dataset)
        with tqdm(valid_dl, unit="batch") as vepoch:
            model.eval()
            with torch.no_grad():
                for x_batch, y_batch in vepoch:
                    vepoch.set_description(f"Epoch {epoch} valid")
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    
                    pred = model(x_batch)
                    loss = loss_fn(pred, y_batch)
                    
                    loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                    is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                    
                    accuracy_hist_valid[epoch] += is_correct.sum()
                    vepoch.set_postfix(loss= loss_hist_valid[epoch].item() /len(valid_dl.dataset),accuracy= 100. *accuracy_hist_valid[epoch].item() / len(valid_dl.dataset))
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
    return loss_hist_train.cpu(), loss_hist_valid.cpu(), accuracy_hist_train.cpu(), accuracy_hist_valid.cpu()

torch.manual_seed(1)
hist = train(model, num_epochs, train_dl, valid_dl, device)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html